In [1]:
from numpy import array
from keras.callbacks import CSVLogger
import numpy as np
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import Embedding
import io

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def load_doc(filename):
    with io.open(filename, 'r', encoding='utf8') as f:
        text = f.read()
    return text

In [4]:
file_path = "/content/drive/My Drive/next-word-prediction/data/data3.txt"
doc = load_doc(file_path)
lines = doc.split('\n')

In [5]:
lines

['a guide to word',
 'guide to word embedding',
 'to word embedding with',
 'word embedding with gensim',
 'embedding with gensim model',
 'with gensim model handson',
 'gensim model handson graph',
 'model handson graph neural',
 'handson graph neural networks',
 'graph neural networks with',
 'neural networks with pytorch',
 'networks with pytorch pytorch',
 'with pytorch pytorch geometric',
 'pytorch pytorch geometric how',
 'pytorch geometric how to',
 'geometric how to use',
 'how to use in',
 'to use in python',
 'use in python databricks',
 'in python databricks how',
 'python databricks how to',
 'databricks how to save',
 'how to save files',
 'to save files in',
 'save files in csv',
 'files in csv on',
 'in csv on your',
 'csv on your local',
 'on your local computer',
 'your local computer a',
 'local computer a stepbystep',
 'computer a stepbystep implementation',
 'a stepbystep implementation of',
 'stepbystep implementation of gradient',
 'implementation of gradient desc

In [6]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

In [7]:
vocab_size

7761

In [8]:
sequences

[[3, 58, 2, 400],
 [58, 2, 400, 1409],
 [2, 400, 1409, 11],
 [400, 1409, 11, 2310],
 [1409, 11, 2310, 86],
 [11, 2310, 86, 3472],
 [2310, 86, 3472, 674],
 [86, 3472, 674, 69],
 [3472, 674, 69, 90],
 [674, 69, 90, 11],
 [69, 90, 11, 301],
 [90, 11, 301, 301],
 [11, 301, 301, 1731],
 [301, 301, 1731, 5],
 [301, 1731, 5, 2],
 [1731, 5, 2, 56],
 [5, 2, 56, 7],
 [2, 56, 7, 35],
 [56, 7, 35, 3473],
 [7, 35, 3473, 5],
 [35, 3473, 5, 2],
 [3473, 5, 2, 232],
 [5, 2, 232, 1410],
 [2, 232, 1410, 7],
 [232, 1410, 7, 2311],
 [1410, 7, 2311, 17],
 [7, 2311, 17, 8],
 [2311, 17, 8, 855],
 [17, 8, 855, 233],
 [8, 855, 233, 3],
 [855, 233, 3, 856],
 [233, 3, 856, 434],
 [3, 856, 434, 4],
 [856, 434, 4, 1732],
 [434, 4, 1732, 1173],
 [4, 1732, 1173, 6],
 [1732, 1173, 6, 1411],
 [1173, 6, 1411, 20],
 [6, 1411, 20, 174],
 [1411, 20, 174, 105],
 [20, 174, 105, 2],
 [174, 105, 2, 401],
 [105, 2, 401, 9],
 [2, 401, 9, 13],
 [401, 9, 13, 302],
 [9, 13, 302, 1412],
 [13, 302, 1412, 266],
 [302, 1412, 266, 3474]

In [9]:
length = 4

In [10]:
print(len(sequences))
print(sequences[0][0])

48546
3


In [11]:
sequences = array(sequences)
sequences

array([[   3,   58,    2,  400],
       [  58,    2,  400, 1409],
       [   2,  400, 1409,   11],
       ...,
       [   2,   55,    3,   74],
       [  55,    3,   74,   47],
       [   3,   74,   47,  673]])

In [12]:
c=1
h=0
while(c!=0):
    limit = len(sequences)
    d =[]
    c=0
    for i in range(0 , limit):
        l=len(sequences[i])
        if(l!=length):
            print(sequences[i])
            sequences = np.delete(sequences, i, 0)
            
            c=1
            print("error at ", i)
            print("Deleting :_________________________ ") 
            h = h+1
            break        
print("Total deletion: ", h)

Total deletion:  0


In [13]:
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [14]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(GRU(100, return_sequences=True))
model.add(GRU(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
csv_logger = CSVLogger('training_1_1.log')
model.fit(X, y, batch_size=500, epochs=1000, callbacks=[csv_logger])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 50)             388050    
                                                                 
 gru (GRU)                   (None, 3, 100)            45600     
                                                                 
 gru_1 (GRU)                 (None, 100)               60600     
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 7761)              783861    
                                                                 
Total params: 1,288,211
Trainable params: 1,288,211
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/1000
98/98 [=====================

In [15]:
model.save('/content/drive/My Drive/next-word-prediction/models/model_3.h5')
dump(tokenizer, open(r'/content/drive/My Drive/next-word-prediction/models/tokenizer_3.pkl', 'wb'))